In [ ]:
from google.colab import files
files.upload()


Saving invoices.csv to invoices (2).csv


{'invoices (2).csv': b'first_name,last_name,email,product_id,qty,amount,invoice_date,address,city,stock_code,job\r\nCarmen Nixon,Todd Anderson,marvinjackson@example.com,133,9,14.57,10/09/1982,283 Wendy Common,West Alexander,36239634,Logistics and distribution manager\r\nMrs. Heather Miller,Julia Moore,jeffrey84@example.net,155,5,65.48,03/10/2012,13567 Patricia Circles Apt. 751,Andreamouth,2820163,Osteopath\r\nCrystal May,Philip Moody,ugoodman@example.com,151,9,24.66,23/03/1976,6389 Debbie Island Suite 470,Coxbury,27006726,Economist\r\nBobby Weber,Mark Scott,ssanchez@example.com,143,4,21.34,17/08/1986,6362 Ashley Plaza Apt. 994,Ninaland,83036521,Sports administrator\r\nKristen Welch,David David,cynthia66@example.net,168,2,83.9,11/06/1996,463 Steven Cliffs Suite 757,Isaiahview,80142652,Chief Marketing Officer\r\nJamie Garcia,Jonathan Tate,luis57@example.net,192,6,55.67,01/09/2016,54443 Kyle Haven Apt. 230,Millsside,53234087,"Therapist, sports"\r\nAmanda Hicks,Stephanie Huff,hendersontrac

In [ ]:
import os
os.listdir()


['.config',
 'data',
 'invoices (2).csv',
 'invoices.csv',
 'invoice_documents.csv',
 'invoices (1).csv',
 'sample_data']

In [ ]:
import pandas as pd

df_invoice = pd.read_csv("invoices.csv")
df_invoice.head()


,first_name,last_name,email,product_id,qty,amount,invoice_date,address,city,stock_code,job
0,Carmen Nixon,Todd Anderson,marvinjackson@example.com,133,9,14.57,10/09/1982,283 Wendy Common,West Alexander,36239634,Logistics and distribution manager
1,Mrs. Heather Miller,Julia Moore,jeffrey84@example.net,155,5,65.48,03/10/2012,13567 Patricia Circles Apt. 751,Andreamouth,2820163,Osteopath
2,Crystal May,Philip Moody,ugoodman@example.com,151,9,24.66,23/03/1976,6389 Debbie Island Suite 470,Coxbury,27006726,Economist
3,Bobby Weber,Mark Scott,ssanchez@example.com,143,4,21.34,17/08/1986,6362 Ashley Plaza Apt. 994,Ninaland,83036521,Sports administrator
4,Kristen Welch,David David,cynthia66@example.net,168,2,83.90,11/06/1996,463 Steven Cliffs Suite 757,Isaiahview,80142652,Chief Marketing Officer


In [ ]:
df_invoice.columns


Index(['first_name', 'last_name', 'email', 'product_id', 'qty', 'amount',
       'invoice_date', 'address', 'city', 'stock_code', 'job'],
      dtype='object')

In [ ]:
import pandas as pd

df = pd.read_csv("invoices.csv")

def row_to_invoice_text(row):
    text = f"""
    Invoice Details:
    Customer Name: {row['first_name']} {row['last_name']}
    Email: {row['email']}
    Product ID: {row['product_id']}
    Quantity: {row['qty']}
    Amount: {row['amount']}
    Invoice Date: {row['invoice_date']}
    Address: {row['address']}, {row['city']}
    Stock Code: {row['stock_code']}
    Job: {row['job']}
    """
    return text

df['text'] = df.apply(row_to_invoice_text, axis=1)
df['label'] = 'invoice'

df_invoice = df[['text', 'label']]
df_invoice.head()


,text,label
0,\n Invoice Details:\n Customer Name: Car...,invoice
1,\n Invoice Details:\n Customer Name: Mrs...,invoice
2,\n Invoice Details:\n Customer Name: Cry...,invoice
3,\n Invoice Details:\n Customer Name: Bob...,invoice
4,\n Invoice Details:\n Customer Name: Kri...,invoice


In [ ]:
df_invoice.shape


(10000, 2)

In [ ]:
df_invoice.to_csv("invoice_documents.csv", index=False)


In [ ]:
df_invoice.head()


,text,label
0,\n Invoice Details:\n Customer Name: Car...,invoice
1,\n Invoice Details:\n Customer Name: Mrs...,invoice
2,\n Invoice Details:\n Customer Name: Cry...,invoice
3,\n Invoice Details:\n Customer Name: Bob...,invoice
4,\n Invoice Details:\n Customer Name: Kri...,invoice


In [ ]:
import re
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()          # 👈 NO word_tokenize
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

df["clean_text"] = df["text"].apply(preprocess)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df["clean_text"].head()


,clean_text
0,invoice details customer name carmen nixon tod...
1,invoice details customer name mrs heather mill...
2,invoice details customer name crystal may phil...
3,invoice details customer name bobby weber mark...
4,invoice details customer name kristen welch da...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df["clean_text"])


In [ ]:
import numpy as np

def identify_invoice_nlp(text):
    clean = preprocess(text)
    vec = vectorizer.transform([clean])
    similarity = np.max((vec @ tfidf_matrix.T).toarray())
    return "invoice" if similarity > 0.1 else "unknown"


In [ ]:
identify_invoice_nlp(df["text"][0])


'invoice'

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_invoice(text):
    return summarizer(
        text[:1000],
        max_length=120,
        min_length=40,
        do_sample=False
    )[0]["summary_text"]


Device set to use cpu


In [ ]:
print(summarize_invoice(df["text"][0]))


Invoice Details
    Customer Name Carmen Nixon Todd Anderson
    Email marvinjackson@example.com
    Product 133
    Quantity 9
    Amount 14.57
    Invoice Date 10/09/1982
    Address 283 Wendy Common West Alexander
    Job Logistics and distribution manager


In [ ]:
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6"
)


Device set to use cpu


In [ ]:
len(df)


10000

In [ ]:
sample_text = df["text"].iloc[0]


In [ ]:
def invoice_nlp_pipeline(text):
    doc_type = identify_invoice_nlp(text)
    summary = summarize_invoice(text)
    return doc_type, summary


In [ ]:
sample_text = df["text"].iloc[0]

doc_type, summary = invoice_nlp_pipeline(sample_text)

print("Document Type:", doc_type)
print("Summary:", summary)


Document Type: invoice
Summary: Invoice Details
    Customer Name Carmen Nixon Todd Anderson
    Email marvinjackson@example.com
    Product 133
    Quantity 9
    Amount 14.57
    Invoice Date 10/09/1982
    Address 283 Wendy Common West Alexander
    Job Logistics and distribution manager
